Currents as a function
====================

We want to calculate the currents in sfTrident as a function of the spins and the Lorentz index.
The result would be a class which precomputes the kinematic (spinors etc) and returns a function.

TODO
-------
- implement J1, J2, J3 and test it
- generalize the test (build a function to do the test -> def test(J0func,resJ0fromClass,...))

In [ ]:
import sftrident as sf
import sftrident.qft as qft
from sftrident.kinutility import laserPolarisation
import numpy as np
import time
times = {}

The current classes
-----------

In [ ]:
class defaultCurrent():
    gammaMu = qft.GammaMatrix()
    
    config = {
        'mass':1.0,
        'a0':1e-4,
        'mode':'bw'
    }
    avialModes = {
        'c': qft.SpinorU,
        'bw': qft.SpinorV
    }
    def __init__(self,q1,q2,k,**kwargs):
        """
        mom ... object of the momenta class
        """
        self.config.update(kwargs)
        self.q1 = q1
        self.q2 = q2
        self.k = k
        self.mode = self.config['mode']
        if not(self.mode in self.avialModes.keys()):
            raise ValueError("There is no build mode called: <%s> !"%(self.mode))
        
        self.mass = self.config['mass']
        self.a0 = self.config['a0']
        self.buildSpinors(self.mode)
        self.polBase =[laserPolarisation(item) for item in [1,2]]
    
    def buildSpinors(self,mode):
        self.sp1 = np.array([qft.SpinorUBar((self.q1,self.mass),s) for s in (1,2)])
        self.sp2 = np.array([self.avialModes[mode]((self.q2,self.mass),s) for s in (1,2)])

class J0(defaultCurrent):
    def __init__(self,q1,q2,k,**kwargs):
        defaultCurrent.__init__(self,q1,q2,k,**kwargs)
    
    def __call__(self,mu,s1,s2):
        return self.sp1[s1]*(self.gammaMu[mu]*self.sp2[s2])

class J1(defaultCurrent):
    def __init__(self,q1,q2,k,**kwargs):
        defaultCurrent.__init__(self,q1,q2,k,**kwargs)
    
    def __call__(self,mu,s1,s2):
        return 0

Test $J_0$
==

In [ ]:
def BuildGrid(ss,p1x,p1y,p1m,p2x,p2y,p2m):
    SS, P1x, P1y, P1m, P2x, P2y, P2m = np.meshgrid(ss,p1x,p1y,p1m,p2x,p2y,p2m,indexing='ij')
    return [SS,P1x,P1y,P1m,P2x,P2y,P2m]

ssInit = np.array([3.353])
#p1_x = np.linspace(0.0,3.5,3)
p1_x = np.random.uniform(0.0,3.5,10)
#print "p1_x: %s"%p1_x
#p1_y = np.linspace(0.0,3.5,3)
p1_y = np.random.uniform(0.0,3.5,10)
#print "p1_y: %s"%p1_y
#p1_m = np.linspace(0.0,3.5,20)
p1_m = np.random.uniform(0.0,3.5,20)
#print "p1_m: %s"%p1_m

#p2_x = np.linspace(0.0,3.5,3)
p2_x = np.random.uniform(0.0,3.5,3)
#print "p2_x: %s"%p2_x
#p2_y = np.linspace(0.0,3.5,3)
p2_y = np.random.uniform(0.0,3.5,3)
#print "p2_y: %s"%p2_y
#p2_m = np.linspace(-1.0,3.5,20)
p2_m = np.random.uniform(-1.0,3.5,20)
#print "p2_m: %s"%p2_m
start = time.time()
kinGridRaw = BuildGrid(ssInit,p1_x,p1_y,p1_m,p2_x,p2_y,p2_m)
end = time.time() - start
times['mesh'] = end
for i,el in enumerate(kinGridRaw):
    print "coord %d: %s"%(i,str(el.shape))

def omegaFromSS(ss):
    #lab
    return (ss**2 - 1.0)/2.0

#initial particles

start = time.time()
omega = omegaFromSS(kinGridRaw[0])
end =  time.time() -start
times['om']=end
print "omega shape: %s"%(str(omega.shape))
#print "omega: %s"%omega

start =  time.time()
E = np.ones(omega.shape)
end =  time.time() - start
times['E'] = end
print "E shape: %s"%(str(E.shape))
#print "E: %s"%E

Et = omega + E
pt = omega
print "Et shape: %s"%(str(Et.shape))
#print "Et: %s"%Et
print "pt shape: %s"%(str(pt.shape))
#print "pt: %s"%pt
start =  time.time()
#P=qft.MinkowskiVector([E,np.zeros(E.shape),np.zeros(E.shape),np.zeros(E.shape)])
P=qft.MinkowskiVector([E,0.0,0.0,0.0])
end =  time.time() - start
times['Pfirst']=end

start =  time.time()
P3m = 0.5*(P._0() - P._3()) - kinGridRaw[3] - kinGridRaw[6]
P3x =  - kinGridRaw[1] - kinGridRaw[4]
P3y =  - kinGridRaw[2] - kinGridRaw[5]
end =  time.time() - start
times['p3coord'] = end

print "P3m shape: %s"%(str(P3m.shape))
print "P3x shape: %s"%(str(P3x.shape))
print "P3y shape: %s"%(str(P3y.shape))


def physArea(P1m,P2m,P3m):
    return (P1m>0) * (P2m>0) * (P3m>0) 


def BuildFinalMom(px,py,pm):
    #print pm
    pp = (px**2 + py**2 + 1)/(4.0*pm)
    return qft.MinkowskiVector(qft.parray([pp+pm,px,py,pp-pm]))

def buildAllMom(p1x,p1y,p1m,p2x,p2y,p2m,p3x,p3y,p3m):
    physAreaArr = physArea(p1m,p2m,p3m)
    print "physArea: %s"%(str(physAreaArr.shape))
    #print physAreaArr.all()==False
    p1xT = p1x[physAreaArr]
    p1yT = p1y[physAreaArr]
    p1mT = p1m[physAreaArr]
    #print "p1m: %s"%(str(p1m.shape))
    #print "p1mT: %s"%(str(p1mT.shape))
    p2xT = p2x[physAreaArr]
    p2yT = p2y[physAreaArr]
    p2mT = p2m[physAreaArr]
    #print "p2m: %s"%(str(p2m.shape))
    #print "p2mT: %s"%(str(p2mT.shape))
    p3xT = p3x[physAreaArr]
    p3yT = p3y[physAreaArr]
    p3mT = p3m[physAreaArr]
    print "p3m: %s"%(str(p3m.shape))
    print "p3mT: %s"%(str(p3mT.shape))
    P1 = BuildFinalMom(p1xT,p1yT,p1mT)
    P2 = BuildFinalMom(p2xT,p2yT,p2mT)
    P3 = BuildFinalMom(p3xT,p3yT,p3mT)
    return P1,P2,P3

start =  time.time()
Pa,Pb,Pc = buildAllMom(kinGridRaw[1],kinGridRaw[2],kinGridRaw[3],kinGridRaw[4],kinGridRaw[5],kinGridRaw[6],P3x,P3y,P3m)
end =  time.time() - start
times['finalMom'] = end

def buildInitMom(om,e,p1x,p1y,p1m,p2x,p2y,p2m,p3x,p3y,p3m):
    physAreaArr = physArea(p1m,p2m,p3m)
    omegaT = om[physAreaArr]
    ET = e[physAreaArr]
    #P=qft.MinkowskiVector([ET,np.zeros(ET.shape),np.zeros(ET.shape),np.zeros(ET.shape)])
    P=qft.MinkowskiVector([ET,0.0,0.0,0.0])
    K = qft.MinkowskiVector([omegaT,np.zeros(omegaT.shape),np.zeros(omegaT.shape),omegaT])
    return K,P

start =  time.time()
Ktest,Ptest = buildInitMom(omega,E,kinGridRaw[1],kinGridRaw[2],kinGridRaw[3],kinGridRaw[4],kinGridRaw[5],kinGridRaw[6],P3x,P3y,P3m)
end =  time.time() - start
times['initMom'] = end
print Ktest.shape
print Ptest.shape

In [ ]:

start = time.time()
J0obj = J0(Pa,Ptest,Ktest,mode='c')
times['initJ0'] = time.time() - start

start = time.time()
res1 = J0obj(2,1,1)
times['calcJ0'] = time.time() - start

calls = int(res1.shape[0])

print "no. calls: %d"%(calls)
print "time init: %1.2e (%1.2e  per call)"%(times['initJ0'],times['initJ0']/float(calls))
print "time calc: %1.2e (%1.2e  per call)"%(times['calcJ0'],times['calcJ0']/float(calls))

compare to serial
-------

In [ ]:
import itertools
SQkinPara=[t for t in itertools.product(ssInit,p1_x,p1_y,p1_m,p2_x,p2_y,p2_m)]
sqP = qft.MinkowskiVector([1.0,0.0,0.0,0.0])
ind = 0
err = 0

gammaMu = qft.GammaMatrix()


times['SQinitJ0'] = 0
times['SQcalcJ0'] = 0

for i,el in enumerate(SQkinPara):
    sqP3m = 0.5 - el[3] - el[6]
    sqP3x = - el[1] - el[4]
    sqP3y = - el[2] - el[5]
    if el[3]>0 and el[6] >0 and sqP3m>0:
        sqP1 = BuildFinalMom(el[1],el[2],el[3])
        sqP2 = BuildFinalMom(el[4],el[5],el[6])
        sqP3 = BuildFinalMom(sqP3x,sqP3y,sqP3m)
        
        start = time.time()
        sqU = qft.SpinorU((sqP,1.0),2)
        sqU1b = qft.SpinorUBar((sqP1,1.0),2)
        times['SQinitJ0'] += time.time() -start
        
        start = time.time()
        sqJ0 = sqU1b*(gammaMu[2]*sqU)
        times['SQcalcJ0'] += time.time() -start
        #print "-"*20
        #print "%s\n%s"%(res1[ind],sqJ0)
        if np.isclose(res1[ind],sqJ0).all():
            pass
        else:
            err+=1
        ind +=1

print "Done with %d errors."%err
for el in times.iterkeys():
    print "%s: %1.2e"%(el,times[el])
    print "-"*20